In [22]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [23]:
import functools
%run ../gw/gw_wrapped_compressed.ipynb
%run ../compressed_class_list/compressed_class_list.ipynb
%run chiang_ogw.ipynb

IndexError: list index out of range

IndexError: list index out of range

In [24]:
def experiment(b, k, A, chiang_ogw, debug=False):
    n = 3
    col = 0
    ool = 0
    cor = 0
    oor = 0 
    
    for t in A.bipart():
        I1 = t[0]
        I2 = t[1]
        bipart_coeff = ClassList.bipart_coeff(A, I1) 

        for i in range(0, n+1):
            for b1 in range(1, b):
                b_ = 0
                if (b - b1) % 4 == 0:
                    b_ = (b - b1) / 4
                else:
                    continue
            
                col += bipart_coeff \
                    * gw_invariant(n, b_, [i, 2, 3] + I1) \
                    * chiang_ogw.chiang_ogw_invariant(n, b1, k, [2, n - i] + I2)
            
        for b1 in range(0, b+1):
            b2 = b - b1
            for k1 in range(0, k+1):
                k2 = k - k1
                ool += bipart_coeff*binomial(k, k1) \
                    * chiang_ogw.chiang_ogw_invariant(n, b1, k1+1, [2] + I1) \
                    * chiang_ogw.chiang_ogw_invariant(n, b2, k2, [2, 3] + I2)
                if debug:
                    print(f"ool, ool_1:{chiang_ogw.chiang_ogw_invariant(n, b1, k1+1, [2] + I1)}\n" +\
                          f"     ool_2: {chiang_ogw.chiang_ogw_invariant(n, b2, k2, [2, 3] + I2)}")

        
        for i in range(0, n+1):
            for b1 in range(1, b):
                b_ = 0
                if (b - b1) % 4 == 0:
                    b_ = (b - b1) / 4
                else:
                    continue
                
                cor += bipart_coeff \
                    * gw_invariant(n, b_, [i, 2, 2] + I1)\
                    * chiang_ogw.chiang_ogw_invariant(n, b1, k, [n-i, 3] + I2)
                
        for b1 in range(0, b+1):
            b2 = b - b1
            for k1 in range(0, k+1):
                k2 = k - k1
                
                oor += bipart_coeff*binomial(k, k1) \
                    * chiang_ogw.chiang_ogw_invariant(n, b1, k1, [2, 2] + I1)\
                    * chiang_ogw.chiang_ogw_invariant(n, b2, k2+1, [3] + I2)
                if debug:
                    print(f"oo3, ool_1:{chiang_ogw.chiang_ogw_invariant(n, b1, k1, [2,2] + I1)}\n" +\
                          f"     ool_2:{chiang_ogw.chiang_ogw_invariant(n, b2, k2+1, [3] + I2)}")

    #print(f"b:{b}, k:{k}, A:{A}, lhs:{col - ool}, rhs:{cor - oor}")
    return (col - ool, cor - oor)



                
        

In [25]:
def get_l1(n, b, k, l2, A1, A2):
    m = n - 3 + 2*b + k + 2*l2 - k*n - 2*l2 * A2 # degree axiom
    if (m % (2*(A1 - 1))) == 0:
        return int(m / (2*(A1 - 1)))
    else:
        -1

In [26]:
# Test 1: compute Chiang OGW Invariant with no boundary points i.e k = 0
chiang_ogw = ChiangOGWInvariant(debug=False)
for b in range(0, 10):
    for l2 in [1, 3, 5]:
        A1 = 2
        A2 = 3
        l1 = get_l1(3, b, 0, l2, A1, A2)
        if l1 < 0 :
            continue
        A = [A1]*l1 + [A2]*l2
        chiang = chiang_ogw.chiang_ogw_invariant(3, b, 0, ClassList.compress(A, 3))
        print(f"b: {b}, A: {A}, chiang ogw: {chiang}")

b: 2, A: [3], chiang ogw: 1
b: 3, A: [2, 3], chiang ogw: -3/8
b: 4, A: [2, 2, 3], chiang ogw: 331/928
b: 5, A: [2, 2, 2, 3], chiang ogw: -25911/59392
b: 6, A: [2, 2, 2, 2, 3], chiang ogw: 83871991/99897344
b: 6, A: [3, 3, 3], chiang ogw: -2994/24389
b: 7, A: [2, 2, 2, 2, 2, 3], chiang ogw: -22125020703/12786860032
b: 7, A: [2, 3, 3, 3], chiang ogw: -36043/780448
b: 8, A: [2, 2, 2, 2, 2, 2, 3], chiang ogw: 133896663195/42006833152
b: 8, A: [2, 2, 3, 3, 3], chiang ogw: -12833319/164089192
b: 9, A: [2, 2, 2, 2, 2, 2, 2, 3], chiang ogw: 9201115703455809/2752959817449472
b: 9, A: [2, 2, 2, 3, 3, 3], chiang ogw: 145655741861/168027332608


In [27]:
# Test: compute Chiang OGW Invariant for a range of values

chiang_ogw1 = ChiangOGWInvariant(debug=False, ogw_2_0_g3=1, ogw_1_1=3)
chiang_ogw2 = ChiangOGWInvariant(debug=False, ogw_2_0_g3=-1, ogw_1_1=3)
chiang_ogw3 = ChiangOGWInvariant(debug=False, ogw_2_0_g3=-1, ogw_1_1=-3)
chiang_ogw4 = ChiangOGWInvariant(debug=False, ogw_2_0_g3=1, ogw_1_1=-3)

for b in range(1, 10):
    for k in range(0, 7):
        for l2 in range(0, 7):
            A1 = 2
            A2 = 3
        
            l1 = get_l1(3, b, k, l2, A1, A2)
            if l1 < 0 :
                continue
            A = [A1]*l1 + [A2]*l2
            
            ogw1 = chiang_ogw1.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))
            ogw2 = chiang_ogw2.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))
            ogw3 = chiang_ogw3.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))
            ogw4 = chiang_ogw4.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))
            
            print(f"b:{b}, k:{k}, A:{A}, ogw_2_0_g3=1, ogw_1_1=3\nogw1:{ogw1}")
            print(f"b:{b}, k:{k}, A:{A}, ogw_2_0_g3=-1, ogw_1_1=3\nogw2:{ogw2}")
            print(f"b:{b}, k:{k}, A:{A}, ogw_2_0_g3=-1, ogw_1_1=-3\nogw3:{ogw3}")
            print(f"b:{b}, k:{k}, A:{A}, ogw_2_0_g3=1, ogw_1_1=-3\nogw4:{ogw4}\n")




b:1, k:0, A:[2], ogw_2_0_g3=1, ogw_1_1=3
ogw1:1/4
b:1, k:0, A:[2], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:1/4
b:1, k:0, A:[2], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:1/4
b:1, k:0, A:[2], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:1/4

b:1, k:1, A:[], ogw_2_0_g3=1, ogw_1_1=3
ogw1:3
b:1, k:1, A:[], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:3
b:1, k:1, A:[], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:-3
b:1, k:1, A:[], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:-3

b:2, k:0, A:[2, 2], ogw_2_0_g3=1, ogw_1_1=3
ogw1:29/64
b:2, k:0, A:[2, 2], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:-35/64
b:2, k:0, A:[2, 2], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:-29/64
b:2, k:0, A:[2, 2], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:35/64

b:2, k:0, A:[3], ogw_2_0_g3=1, ogw_1_1=3
ogw1:1
b:2, k:0, A:[3], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:-1
b:2, k:0, A:[3], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:-1
b:2, k:0, A:[3], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:1

b:2, k:1, A:[2], ogw_2_0_g3=1, ogw_1_1=3
ogw1:-9/16
b:2, k:1, A:[2], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:-9/16
b:2, k:1, A:[2], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:-9/16
b:2, k:1, A:[2]

b:6, k:3, A:[2, 2, 2], ogw_2_0_g3=1, ogw_1_1=3
ogw1:-256188635325/84013666304
b:6, k:3, A:[2, 2, 2], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:-8721/4096
b:6, k:3, A:[2, 2, 2], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:-256188635325/84013666304
b:6, k:3, A:[2, 2, 2], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:-8721/4096

b:6, k:3, A:[2, 3], ogw_2_0_g3=1, ogw_1_1=3
ogw1:-1611150975/1312713536
b:6, k:3, A:[2, 3], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:-45/64
b:6, k:3, A:[2, 3], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:-1611150975/1312713536
b:6, k:3, A:[2, 3], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:-45/64

b:6, k:4, A:[2, 2], ogw_2_0_g3=1, ogw_1_1=3
ogw1:66269832960625/17663873340416
b:6, k:4, A:[2, 2], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:-2003/1024
b:6, k:4, A:[2, 2], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:-66269832960625/17663873340416
b:6, k:4, A:[2, 2], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:2003/1024

b:6, k:4, A:[3], ogw_2_0_g3=1, ogw_1_1=3
ogw1:359559274375/275998020944
b:6, k:4, A:[3], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:-11/16
b:6, k:4, A:[3], ogw_2_0_g3=-1, ogw_1_1=-3


b:8, k:1, A:[2, 2, 2, 2, 2, 2, 2], ogw_2_0_g3=1, ogw_1_1=3
ogw1:4553686902057/84013666304
b:8, k:1, A:[2, 2, 2, 2, 2, 2, 2], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:188733/4096
b:8, k:1, A:[2, 2, 2, 2, 2, 2, 2], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:4553686902057/84013666304
b:8, k:1, A:[2, 2, 2, 2, 2, 2, 2], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:188733/4096

b:8, k:1, A:[2, 2, 2, 2, 2, 3], ogw_2_0_g3=1, ogw_1_1=3
ogw1:257396434007/21003416576
b:8, k:1, A:[2, 2, 2, 2, 2, 3], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:10243/1024
b:8, k:1, A:[2, 2, 2, 2, 2, 3], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:257396434007/21003416576
b:8, k:1, A:[2, 2, 2, 2, 2, 3], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:10243/1024

b:8, k:1, A:[2, 2, 2, 3, 3], ogw_2_0_g3=1, ogw_1_1=3
ogw1:509819257/164089192
b:8, k:1, A:[2, 2, 2, 3, 3], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:5/2
b:8, k:1, A:[2, 2, 2, 3, 3], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:509819257/164089192
b:8, k:1, A:[2, 2, 2, 3, 3], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:5/2

b:8, k:1, A:[2, 3, 3, 3], ogw_2_0_g3=1, ogw_1_1=3
ogw1:75

b:9, k:2, A:[2, 2, 2, 2, 2, 2, 2], ogw_2_0_g3=1, ogw_1_1=3
ogw1:-676445128666978838475/18521913651800047616
b:9, k:2, A:[2, 2, 2, 2, 2, 2, 2], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:-10693903263/1073741824
b:9, k:2, A:[2, 2, 2, 2, 2, 2, 2], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:-676445128666978838475/18521913651800047616
b:9, k:2, A:[2, 2, 2, 2, 2, 2, 2], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:-10693903263/1073741824

b:9, k:2, A:[2, 2, 2, 2, 2, 3], ogw_2_0_g3=1, ogw_1_1=3
ogw1:-2038161211629074175/144702450404687872
b:9, k:2, A:[2, 2, 2, 2, 2, 3], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:-388509/8388608
b:9, k:2, A:[2, 2, 2, 2, 2, 3], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:-2038161211629074175/144702450404687872
b:9, k:2, A:[2, 2, 2, 2, 2, 3], ogw_2_0_g3=1, ogw_1_1=-3
ogw4:-388509/8388608

b:9, k:2, A:[2, 2, 2, 3, 3], ogw_2_0_g3=1, ogw_1_1=3
ogw1:-5361377201995675/1130487893786624
b:9, k:2, A:[2, 2, 2, 3, 3], ogw_2_0_g3=-1, ogw_1_1=3
ogw2:25537/65536
b:9, k:2, A:[2, 2, 2, 3, 3], ogw_2_0_g3=-1, ogw_1_1=-3
ogw3:-5361377201995675/

In [ ]:
# Test: compare two sides of an equation as defined in the function experiment()

chiang_ogw1 = ChiangOGWInvariant(debug=False, ogw_2_0_g3=1, ogw_1_1=3)
chiang_ogw2 = ChiangOGWInvariant(debug=False, ogw_2_0_g3=-1, ogw_1_1=3)
chiang_ogw3 = ChiangOGWInvariant(debug=False, ogw_2_0_g3=-1, ogw_1_1=-3)
chiang_ogw4 = ChiangOGWInvariant(debug=False, ogw_2_0_g3=1, ogw_1_1=-3)


for b in range(1, 10):
    for k in range(0, 7):
        for l2 in range(0, 7):
            for l1 in range(0, 7):
                A1 = 2
                A2 = 3
    
                A = ClassList.compress([A1]*l1 + [A2]*l2, 3)
                lhs1, rhs1 = experiment(b, k, A, chiang_ogw1)
                lhs2, rhs2 = experiment(b, k, A, chiang_ogw2)
                lhs3, rhs3 = experiment(b, k, A, chiang_ogw3)
                lhs4, rhs4 = experiment(b, k, A, chiang_ogw4)
                

                if lhs1 != 0:
                    print(f"b:{b}, k:{k}, A:{A}, ogw_2_0_g3=1, ogw_1_1=3\nlhs1:{lhs1}\nrhs1:{rhs1} \n")
                if lhs2 != 0:
                    print(f"b:{b}, k:{k}, A:{A}, ogw_2_0_g3=-1, ogw_1_1=3\nlhs2:{lhs2}\nrhs2:{rhs2} \n")
                if lhs3 != 0:
                    print(f"b:{b}, k:{k}, A:{A}, ogw_2_0_g3=-1, ogw_1_1=-3\nlhs3:{lhs3}\nrhs3:{rhs3} \n")
                if lhs4 != 0:
                    print(f"b:{b}, k:{k}, A:{A}, ogw_2_0_g3=1, ogw_1_1=-3\nlhs4:{lhs4}\nrhs4:{rhs4} \n")
                
                